In [1]:
# pip install opencv-python

  Using cached opencv_python-4.5.5.62-cp36-abi3-win_amd64.whl (35.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np

In [2]:
img1 = cv2.imread('images.jpeg')
cv2.imshow('abc', img1)
cv2.waitKey(0)

-1

In [3]:
def preprocess(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 1)
#     print(img_blur.shape)
    img_thres = cv2.adaptiveThreshold(img_blur, 255, 1, 1, 17, 5)
    return img_thres

In [4]:
img_thres = preprocess(img1)

In [5]:
def draw_contour(img_original, img_thres):
    contours, heirarchy = cv2.findContours(img_thres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_cont = cv2.drawContours(img_original, contours, -1, (0, 255, 0), 3)
    return img_cont, contours

In [6]:
img_cont, cont = draw_contour(img1, img_thres)

In [7]:
cont

(array([[[155, 182]],
 
        [[154, 183]],
 
        [[154, 190]],
 
        [[155, 191]],
 
        [[156, 190]],
 
        [[156, 188]],
 
        [[157, 187]],
 
        [[158, 187]],
 
        [[159, 186]],
 
        [[160, 186]],
 
        [[161, 185]],
 
        [[162, 186]],
 
        [[162, 190]],
 
        [[163, 191]],
 
        [[164, 190]],
 
        [[164, 189]],
 
        [[163, 188]],
 
        [[163, 187]],
 
        [[165, 185]],
 
        [[167, 187]],
 
        [[166, 188]],
 
        [[166, 190]],
 
        [[168, 190]],
 
        [[169, 189]],
 
        [[171, 191]],
 
        [[172, 191]],
 
        [[173, 190]],
 
        [[174, 190]],
 
        [[174, 189]],
 
        [[175, 188]],
 
        [[177, 190]],
 
        [[180, 190]],
 
        [[181, 191]],
 
        [[183, 191]],
 
        [[185, 189]],
 
        [[185, 185]],
 
        [[184, 184]],
 
        [[183, 184]],
 
        [[182, 183]],
 
        [[181, 184]],
 
        [[179, 184]],
 
        [[177, 1

In [8]:
def biggestContour(cont):
    biggest = np.array([])
    max_area = 0
    for i in cont:
        area = cv2.contourArea(i)
        if area > 25:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02*peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest, max_area

In [9]:
biggest, max_area = biggestContour(cont)
print(biggest, max_area)

[] 0


In [12]:
def reorder(points):
    points = points.reshape((4, 2))
    new_points = np.zeros((4, 1, 2), dtype = np.int32)
    add = points.sum(1)
    new_points[0] = points[np.argmin(add)]
    new_points[3] = points[np.argmax(add)]
    diff = np.diff(points, axis = 1)
    new_points[1] = points[np.argmin(diff)]
    new_points[2] = points[np.argmax(diff)]
    return new_points

In [13]:
print(biggest, reorder(biggest))

[[[162  33]]

 [[164  38]]

 [[164  42]]

 [[170  48]]] [[[162  33]]

 [[162  33]]

 [[164  42]]

 [[170  48]]]


In [14]:
biggest = reorder(biggest)
cv2.drawContours(img_cont, biggest, -1, (0, 0), 15)
pts1 = np.float32(biggest)
pts2 = np.float32([[0, 0], [450, 0], [0, 450], [450, 450]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)
img_wrap_colored = cv2.warpPerspective(img1, matrix, (450, 450))
img_wrap_colored = cv2.cvtColor(img_wrap_colored, cv2.COLOR_BGR2GRAY)
cv2.imshow('abc', img_wrap_colored)
cv2.waitKey(0)

-1